In [ ]:
%matplotlib tk

from datetime import date, datetime, timedelta
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import imageio.v2 as imageio
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

: 

: 

In [ ]:
VMIN = 0
VMAX = 1

SIGNATURE = 'By Will Bradley, 2023' 

In [ ]:
def _get_urls(base_scrape_url: str, start_date: date, end_date: date) -> list[tuple[date, str]]:
    scraping_url_index = -1

    out: list[tuple[date, str]] = []
    for days_delta in range((end_date - start_date).days + 1):
        target_date = start_date + timedelta(days=days_delta)
        while True:
            scraping_url_suffix = "" if scraping_url_index == -1 else f"-{scraping_url_index}"
            scraping_url = base_scrape_url + scraping_url_suffix
            response = requests.get(scraping_url)
            
            try:
                response.raise_for_status()
            except requests.HTTPError:
                print(target_date, scraping_url, response)
                scraping_url_index += 1
                continue

            soup = BeautifulSoup(response.text)

            datetime_element = soup.select_one('time.datetime[datetime]')
            datetime_str: str = datetime_element['datetime']
            date_ = datetime.strptime(datetime_str, r'%Y-%m-%dT%H:%M:%SZ').date()

            if date_ == target_date:
                csv_elements = soup.select("a[type='text/csv']")
                csv_url_part: str = csv_elements[1]['href']
                csv_url = urljoin(scraping_url, csv_url_part)

                out.append((target_date, csv_url))
                break
        scraping_url_index += 1

    return out
    
    

In [ ]:
# Galen Metzger suggested using 2022 ballots returned to normalize instead of voter registration
ballots_2022_df = pd.read_csv('AbsenteeCounts_Muni__25.csv', index_col='HINDI', usecols=['BallotsReturned', 'HINDI'])\
    .rename(columns={'BallotsReturned': 'BallotsReturned2022'})\
    .drop(99999) # TOTAL column

In [ ]:
gdf = gpd.read_file('WI_Cities%2C_Towns_and_Villages_(January_2023)') # https://data-ltsb.opendata.arcgis.com/datasets/2d13492a59a24dd0ba990abf1f86800f_0/explore?location=43.322926%2C-89.269110%2C8.58

In [ ]:
plt.close('all')

In [ ]:
fnames: list[str] = []

In [ ]:
urls_2023 = _get_urls(
    'https://elections.wi.gov/resources/statistics/absentee-ballot-report-april-4-2023-spring-election',
    date(2023, 3, 20),
    date.today()
)

2023-03-29 https://elections.wi.gov/resources/statistics/absentee-ballot-report-april-4-2023-spring-election-8 <Response [403]>


In [ ]:
OUTPUT_DIR = 'output'

In [ ]:
for i, (date_, url) in enumerate(urls_2023):
    ballots_df = pd.read_csv(StringIO(requests.get(url).text))
    voting_merged_df = pd.merge(ballots_df, ballots_2022_df, on='HINDI')
    voting_merged_df['HINDI'] = voting_merged_df['HINDI'].apply(lambda n: str(n).zfill(5))

    merged = gdf.merge(voting_merged_df, left_on='DOA', right_on='HINDI', how='left')
    merged.set_index('GEOID', drop=True, inplace=True)
    
    merged['returned_rate'] = merged['BallotsReturned'] / merged['BallotsReturned2022']

    missing_merge = merged['BallotsReturned'].isna().sum()
    print(
        date_, 
        url,
        "missing because of merge:", missing_merge, 
        "median `returned_rate`:", merged['returned_rate'].median()
    )

    CMAP = plt.get_cmap('YlGn')

    plot = merged.plot(
        'returned_rate', 
        legend=True, 
        cmap=CMAP,
        norm=mpl.colors.BoundaryNorm(np.linspace(VMIN, VMAX, 101), CMAP.N), 
        missing_kwds={'color': 'gray'}, 
        legend_kwds={
            'label': 'Ballots returned as of March 1, 2023,\nnormalized to total (mail) ballots returned in 2022 general', 
            'shrink': 0.7, 
            'ticks': np.linspace(VMIN, VMAX, 11)
        }, 
        lw=0.2, 
        ec='black'
    )
    plt.axis('off')

    fig = plt.gcf()
    fig.set_size_inches(8, 8)
    fig.text(0.05, 0.05, SIGNATURE)
    

    ax = plt.gca()
    ax.set_title(f"Wisconsin 2023 spring election turnout by municipality, {date_.isoformat()}")

    cbar = ax.get_figure().get_axes()[1]
    cbar.minorticks_off()

    fname = f"{OUTPUT_DIR}/early_vote_{date_.isoformat()} (Day {str(i + 1).zfill(2)}).png"
    fig.savefig(fname, transparent=False, dpi=1000, bbox_inches='tight')
    fnames.append(fname)


2023-03-20 https://elections.wi.gov/sites/default/files/documents/AbsenteeCounts_Muni_2023%20Spring%20Election.csv missing because of merge: 92 median `returned_rate`: 0.0
2023-03-21 https://elections.wi.gov/sites/default/files/documents/AbsenteeCounts_Muni_2023%20Spring%20Election_0.csv missing because of merge: 90 median `returned_rate`: 0.0
2023-03-22 https://elections.wi.gov/sites/default/files/documents/AbsenteeCounts_Muni_2023%20Spring%20Election_1.csv missing because of merge: 90 median `returned_rate`: 0.05
2023-03-23 https://elections.wi.gov/sites/default/files/documents/AbsenteeCounts_Muni_2023%20Spring%20Election_2.csv missing because of merge: 88 median `returned_rate`: 0.0975609756097561
2023-03-24 https://elections.wi.gov/sites/default/files/documents/AbsenteeCounts_Muni_2023%20Spring%20Election_3.csv missing because of merge: 88 median `returned_rate`: 0.15
2023-03-25 https://elections.wi.gov/sites/default/files/documents/AbsenteeCounts_Muni_2023%20Spring%20Election_4.cs

In [ ]:
OUT_PATH = f'{OUTPUT_DIR}/election.gif'
durations = [0.75 for _ in fnames]
durations[-1] = 3

with imageio.get_writer(OUT_PATH, mode='I', duration=durations) as writer:
    for fname in fnames:
        image = imageio.imread(fname)
        writer.append_data(image)
        

In [ ]:
TARGET_FILE_SIZE = 5E6
current_file_size = os.path.getsize(OUT_PATH)
shrink_ratio = min(TARGET_FILE_SIZE / current_file_size, 1)
print(shrink_ratio)
os.system(f"convert {OUT_PATH} -scale {shrink_ratio * 100}% {OUT_PATH}_shrunk.gif")

0.1898089811763776


0